In [11]:
import os
import pandas as pd

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

import boto3
role = get_execution_role()

In [12]:
jars = sagemaker_pyspark.classpath_jars()
classpath = ":".join(sagemaker_pyspark.classpath_jars())
spark = (SparkSession.builder
         .config("spark.driver.extraClassPath", classpath)
         .config("spark.executor.memory","60g")
         .config("spark.driver.memory","60g")
         .config("spark.driver.maxResultSize", "0")
         .master("local[*]").getOrCreate())

spark.conf.set("spark.sql.caseSensitive", "true")

In [13]:
spark

In [14]:
bucket = 'politicos-dataset-us-east-1'
region = 'us-east-1'
prefix = 'gpt2_fine_tune/tweets_izquierda/parquet/'
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 's3.{}.amazonaws.com'.format(region))

In [15]:
SDF = spark.read.parquet('s3a://{}/{}'.format(bucket, prefix))

In [16]:
SDF.count()

22571

## Make Datasets

In [17]:
df = SDF.toPandas()
df

,screen_name,text_clean,Partido,Coalicion,dt
0,boris_barrera,Saludamos esta gran Feria de la #MadreTrabajad...,PC,Nueva Mayoría,2019-06-02 19:23:40
1,pablovidalrojas,"Patipelado, da: \n1. Personas que no tienen ac...",RD,Frente Amplio,2019-06-05 21:38:46
2,danieljadue,@RebecaGaeteS @jorge_ulloa @spinera Quien es @...,PC,Nueva Mayoría,2019-06-06 21:52:09
3,KarolCariola,Esta mañana junto al alcalde @Duranbaronti y d...,PC,Nueva Mayoría,2019-06-10 14:28:46
4,camila_vallejo,Frente a la negativa de la ministra Cubillos d...,PC,Nueva Mayoría,2019-06-27 14:43:08
...,...,...,...,...,...
22566,PamJiles,No voy a adjetivar Nietitos porque soy una dam...,PH,Frente Amplio,2020-07-09 14:42:56
22567,daniel_nunez_a,Solidarizo con el diputado Jaime @Muletjaime a...,PC,Nueva Mayoría,2020-07-15 15:00:57
22568,Muletjaime,Mi solidaridad @matiaswalkerp. Gravísimas amen...,FRVS,Nueva Mayoría,2020-07-15 21:39:18
22569,camila_vallejo,Este jueves comenzará discusión d #ImpuestoALo...,PC,Nueva Mayoría,2020-07-18 18:57:45


In [19]:
from sklearn.model_selection import train_test_split
import re

def build_text_files(tweets, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for tweet in tweets:
        text = str(tweet).strip()
        text = re.sub(r"\n", " ", text)
        data += text + "  "
    f.write(data)

In [20]:
tweets = df['text_clean']

In [21]:
train, test = train_test_split(tweets,test_size=0.15)
build_text_files(train,'train_dataset_izq.txt')
build_text_files(test,'test_dataset_izq.txt')
len(train), len(test)

(19185, 3386)

## 